In [11]:
from urllib.request import Request,urlopen
import pandas as pd
from selenium import webdriver
import time

browser = webdriver.Firefox(executable_path='E:/ZProjetos/geckodriver/geckodriver.exe')

browser.get('https://www.gru.com.br/pt/passageiro/descubra-gru/lojas/todos-os-estabelecimentos')

time.sleep(20)

#table = browser.find_element_by_class_name('estabelecimento')
table = browser.find_element_by_id("tabela-0")

table_html = table.get_attribute('outerHTML')

df = pd.read_html(str(table_html))

#print(df)
df = df[0]

In [10]:
df.head(300)

,Alimentação,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,AlimentaçãoConheça todas as opções de restaura...,NaN,NaN,NaN,NaN
1,365 DeliCafés / Sorveterias Terminal 2Conector...,NaN,NaN,NaN,NaN
2,Terminal 2,Conector de Embarque,24h,(11)2445-4637,Área Restrita
3,Terminal 3,Embarque,24h,(11)2445-4637,Área Restrita
4,Terminal 3,Mezanino,24h,(11) 2445-4637,Área Restrita
5,Terminal 2,Píer Oeste,24h,(11) 2445-4637,Área Restrita
6,Terminal 3,Embarque Remoto,06h - 23h,NaN,Área Restrita
7,Amor aos PedaçosCafés / Sorveterias Terminal 2...,NaN,NaN,NaN,NaN
8,Terminal 2,Píer Oeste,07h - 22h,NaN,Área Pública
9,Bacio di LatteCafés / Sorveterias Terminal 2Em...,NaN,NaN,NaN,NaN


In [71]:
df.shape

(214, 5)

In [27]:
df['ALIM'] = df['Alimentação'][(df['Alimentação'].str.contains("Terminal") == False) ]

In [28]:
df

,Alimentação,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Name,ALIM
0,AlimentaçãoConheça todas as opções de restaura...,NaN,NaN,NaN,NaN,AlimentaçãoConheça todas as opções de restaura...,AlimentaçãoConheça todas as opções de restaura...
1,365 DeliCafés / Sorveterias Terminal 2Conector...,NaN,NaN,NaN,NaN,NaN,NaN
2,Terminal 2,Conector de Embarque,24h,(11)2445-4637,Área Restrita,NaN,NaN
3,Terminal 3,Embarque,24h,(11)2445-4637,Área Restrita,NaN,NaN
4,Terminal 3,Mezanino,24h,(11) 2445-4637,Área Restrita,NaN,NaN
5,Terminal 2,Píer Oeste,24h,(11) 2445-4637,Área Restrita,NaN,NaN
6,Terminal 3,Embarque Remoto,06h - 23h,NaN,Área Restrita,NaN,NaN
7,Amor aos PedaçosCafés / Sorveterias Terminal 2...,NaN,NaN,NaN,NaN,NaN,NaN
8,Terminal 2,Píer Oeste,07h - 22h,NaN,Área Pública,NaN,NaN
9,Bacio di LatteCafés / Sorveterias Terminal 2Em...,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
#df['ALIM'] = df['Alimentação'].str[0:df['Alimentação'].str.find("/")]
def position(df):
    
    if df['Alimentação'].str.find("/") > 0: 
        df['Alimentação'].str.find("/") 
    else: 
        100
    
df['ALIM'] = df['Alimentação'].str[0: position(df)  ]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [42]:
df['Alimentação'].str.find("/")

0      -1
1      14
2      -1
3      -1
4      -1
       ..
209    -1
210    -1
211    -1
212    -1
213    -1
Name: Alimentação, Length: 214, dtype: int64

In [41]:
df

,Alimentação,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Name,ALIM
0,AlimentaçãoConheça todas as opções de restaura...,NaN,NaN,NaN,NaN,AlimentaçãoConheça todas as opções de restaura...,Alimentaçã
1,365 DeliCafés / Sorveterias Terminal 2Conector...,NaN,NaN,NaN,NaN,NaN,365 DeliCa
2,Terminal 2,Conector de Embarque,24h,(11)2445-4637,Área Restrita,NaN,Terminal 2
3,Terminal 3,Embarque,24h,(11)2445-4637,Área Restrita,NaN,Terminal 3
4,Terminal 3,Mezanino,24h,(11) 2445-4637,Área Restrita,NaN,Terminal 3
5,Terminal 2,Píer Oeste,24h,(11) 2445-4637,Área Restrita,NaN,Terminal 2
6,Terminal 3,Embarque Remoto,06h - 23h,NaN,Área Restrita,NaN,Terminal 3
7,Amor aos PedaçosCafés / Sorveterias Terminal 2...,NaN,NaN,NaN,NaN,NaN,Amor aos P
8,Terminal 2,Píer Oeste,07h - 22h,NaN,Área Pública,NaN,Terminal 2
9,Bacio di LatteCafés / Sorveterias Terminal 2Em...,NaN,NaN,NaN,NaN,NaN,Bacio di L


In [24]:
df['Name'] = df['Alimentação'][(df['Alimentação'].str.contains("Terminal") == False) ]

In [26]:
df.head(100)

,Alimentação,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Name
0,AlimentaçãoConheça todas as opções de restaura...,NaN,NaN,NaN,NaN,AlimentaçãoConheça todas as opções de restaura...
1,365 DeliCafés / Sorveterias Terminal 2Conector...,NaN,NaN,NaN,NaN,NaN
2,Terminal 2,Conector de Embarque,24h,(11)2445-4637,Área Restrita,NaN
3,Terminal 3,Embarque,24h,(11)2445-4637,Área Restrita,NaN
4,Terminal 3,Mezanino,24h,(11) 2445-4637,Área Restrita,NaN
5,Terminal 2,Píer Oeste,24h,(11) 2445-4637,Área Restrita,NaN
6,Terminal 3,Embarque Remoto,06h - 23h,NaN,Área Restrita,NaN
7,Amor aos PedaçosCafés / Sorveterias Terminal 2...,NaN,NaN,NaN,NaN,NaN
8,Terminal 2,Píer Oeste,07h - 22h,NaN,Área Pública,NaN
9,Bacio di LatteCafés / Sorveterias Terminal 2Em...,NaN,NaN,NaN,NaN,NaN
